## Part 1: Events
Events allow us to handle user input or other changes in state that should affect our program. We have already seen an example of an event in the last section. In the game loop there was a condition looking to see if the window had been closed. If the window was closed, we wanted to exit out program. 

Handling user input through the mouse and keyboard works in a very similar fasion. Pygame maintains a list of events that have occured since we last checked. We can read through this list looking for anything we are interested in. 

In the example below we look for user input events and log them to the terminal.

In [1]:
import pygame
from pygame.locals import *

pygame.init()
clock = pygame.time.Clock()
screen = pygame.display.set_mode((800,600))

running=True
while running:
    for event in pygame.event.get():
        if event.type==QUIT:
            running=False
            
            
        #Here we are looking for all KEYDOWN events
        if event.type==KEYDOWN:
            #we can access key data like this
            # (event.key is an int but we can turn it into a character by adding a chr() around it)
            print(chr(event.key))
        
        #This finds all MOUSEBUTTONDOWN events
        if event.type==MOUSEBUTTONDOWN:
            #we can access the location of the mouse click like this
            print(event.pos)
        clock.tick(30)
pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
q
w
e
r
t
t
y
u
i
o
(197, 408)
(318, 295)
(430, 423)
(526, 115)


KeyboardInterrupt: 

NOTICE: chr() doesn't handle some of the keys on your keyboard properly (remember event.key is an int). This is because chr is only for character so the 'backspace' or 'left' key don't get translated right. We still might want to talk about these keys though. We can use preset variables to access these values without having to remember them. [Here](https://www.pygame.org/docs/ref/key.html) is a list of these variable names. The general pattern is 'K_' and then the name of the key in all capitals. For example K_BACKSPACE or K_LEFT.

We only looked at two events (KEYDOWN and MOUSEBUTTONDOWN) but there are many more. There are also KEYUP and MOUSEBUTTONUP events which occur when keys and buttons are released (rather than pressed). There are also MOUSEMOTION events for anytime the mouse is moved across the window and are events that log when the size of the window is changed (so we know when to redraw). We have already seen the QUIT event that occurs when the window is closed.

## Part 2: Sprites
We are going to use what we learned about classes and objects to make building our game easier. Pygame provides us a class called sprites which will allow us to draw game pieces and calculate collisions between them. 

#### Sprites Example

In [3]:
import pygame
from pygame.locals import *

clock=pygame.time.Clock()

pygame.init()

#we are creating a player class which inherits from the sprite class
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super(Player, self).__init__()
        #The player has a surface attribute which tells how big it is
        self.surf = pygame.Surface((75,75))
        self.surf.fill((255,255,255))
        #initial position of the player
        self.rect = self.surf.get_rect(center=(300,400))
        #direction for the sprite to move
        self.direction=(1,0)
        self.speed = 5
        
    def updatePos(self):
        self.rect.move_ip(self.direction[0]*self.speed, self.direction[1]*self.speed)
        
screen = pygame.display.set_mode((800,600))

#Create a new player
player = Player()

running=True
while running:
    for event in pygame.event.get():
        if event.type==QUIT:
            running=False
        
        #Logic for changing player.direction
        if event.type==KEYUP:
            if event.key == K_UP:
                player.direction = (0,-1)
            if event.key == K_DOWN:
                player.direction = (0,1)
            if event.key == K_RIGHT:
                player.direction = (1,0)
            if event.key == K_LEFT:
                player.direction = (-1,0)
    
    #updates player position, changes based on player direction
    player.updatePos()
    
    # Draw player
    screen.fill((0,0,0))
    screen.blit(player.surf, player.rect)
    
    # Redraw the screen
    pygame.display.flip()

    clock.tick(50)
    
pygame.quit()

### Challenges:
* Try playing with the animation rate and speed of the player to see how they affect the box's motion
* Allow the box to be controlled by other keys (W,A,S,D perhaps?)
* Add constraints on the box's position so it cannot leave the screen


## Detecting Collisions
We can use some of pygames tools to detect collisions between our sprites. To test this out let's create an enemy that chases our player, and we'll figure out when it catches us using the collision tools. 

In Step 1 we define an enemy class that looks a lot like the player class, only it has a very different update function. Rather than accepting input from the user it simply points itself in the direction of the white box.

In Step 2 we create groups of sprites. These groups allow us to detect collisions between sprites. We create an 'enemies' group with our enemy sprite in it to test when it collides with our player (STEP 4). We also have a all_sprites group that gives us something to iterate over as we render (rather than having to render each sprite by name)

Step 3: We update the positions of the sprites and redraw them

Step 4: We check that our player has not collided with any of the sprites in the enemies group. Checking for collisions can actually be pretty hard from a programming perspective and can take a lot of computing resources if its not done efficiently. 
The function we are useing, 'sprite.pygame.spritecollideany', takes two arguments, one sprite and one group of sprites, and tells us if the one sprite has collided with any in the group. We use it to detect if our player has collided with an enemy. However, it would not tell us if two enemies collided. 


In [4]:
import pygame
from pygame.locals import *

clock=pygame.time.Clock()

pygame.init()

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super(Player, self).__init__()
        self.surf = pygame.Surface((75,75))
        self.surf.fill((255,255,255))
        self.rect = self.surf.get_rect(center=(300,400))
        self.direction=(1,0)
        self.speed=10
        
    def update(self):
        self.rect.move_ip(self.direction[0]*self.speed, self.direction[1]*self.speed)
        
# STEP 1: Creates an enemy which tries to catch the player
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super(Enemy, self).__init__()
        self.surf = pygame.Surface((20,10))
        self.surf.fill((255,0,0))
        self.rect = self.surf.get_rect(center=(10,5))
        self.speed = 7
    
    #Logic to follow the player
    def update(self,player):
        x_diff = player.rect[0] - self.rect[0]
        y_diff = player.rect[1] - self.rect[1]
        mag = (x_diff**2+y_diff**2)**(1/2)
        direction = (self.speed/mag*x_diff, self.speed/mag*y_diff)
        self.rect.move_ip(direction[0], direction[1])

screen = pygame.display.set_mode((800,600))

#STEP 2: creating sprites and sprite groups

#Create a new player
player = Player()

#create an enemy
enemy = Enemy()

#Create enemy group (used for collisions)
enemies = pygame.sprite.Group()
enemies.add(enemy)

#creates group for all sprites (used for drawing)
all_sprites = pygame.sprite.Group()
#add player
all_sprites.add(player)
#add enemy
all_sprites.add(enemy)


running=True
while running:
    for event in pygame.event.get():
        if event.type==QUIT:
            running=False
        if event.type==KEYUP:
            if event.key == K_UP:
                player.direction = (0,-1)
            if event.key == K_DOWN:
                player.direction = (0,1)
            if event.key == K_RIGHT:
                player.direction = (1,0)
            if event.key == K_LEFT:
                player.direction = (-1,0)
    
    #STEP 3: update positions
    player.update()
    enemy.update(player)
    screen.fill((0,0,0))
    for entity in all_sprites:
        screen.blit(entity.surf, entity.rect)
    
    #STEP 4: test for collisions
    if pygame.sprite.spritecollideany(player, enemies):
        player.kill()
        running=False
    
    pygame.display.flip()
    clock.tick(30)
    
pygame.quit()


### Challenges
* Play with the relative speeds of the player, enemy, fps rate. See how it changes the game.
* Try making more enemies with different colors, speeds, shapes, and starting positions
* Add walls for your player to hide behind. Hint: create a wall sprite
